In [1]:
import pickle
import urllib.request as req
import pandas as pd
import numpy as np
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import itertools

In [4]:
from keras import backend as K

Using TensorFlow backend.


In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
#hamming loss
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [7]:
def download_images(dataframe,out_folder):
    ids = dataframe.index.values
    for image_id in ids:
        url = dataframe.loc[image_id]['image_url']
        if os.path.isfile(out_folder + '/' + str(image_id) + '.jpg') == False:
            r = req.urlretrieve(url, out_folder + '/' + str(image_id) + '.jpg')
    return True

In [8]:
def return_image_name_col(df):
    ids = df.index.values
    names = []
    for i in ids:
        names.append(str(i)+'.jpg')
    return np.array(names)

In [9]:
#load training set
pickle_in_train = open("all_books_train.pickle","rb")
train = pickle.load(pickle_in_train)

In [10]:
#load test set
pickle_in_test = open("all_books_test.pickle","rb")
test = pickle.load(pickle_in_test)

In [11]:
train.shape

(36389, 14)

In [12]:
test.shape

(12131, 14)

In [13]:
#show training data
train.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,genres_list,genres_cut
6377,Nicole Jacquelyn,"""How is it, that someone can make decision aft...",NaN,Kindle Edition,NaN,305 pages,4.02,17962,1159,Craving Constellations,Romance|Romance|Contemporary Romance|Contemporary,https://images.gr-assets.com/books/1377563339l...,"[Romance, Contemporary, Contemporary Romance]","[Romance, Contemporary]"
19880,Adam Schell,A village in Tuscany is the setting for this j...,NaN,Hardcover,9.78039E+12,340 pages,3.74,646,161,"Tomato Rhapsody: A Fable of Love, Lust & Forbi...",Fiction|Historical|Historical Fiction|Cultural...,https://images.gr-assets.com/books/1320422913l...,"[Historical Fiction, Fiction, Food and Drink, ...","[Fiction, Romance, Historical, Cultural]"
23352,Robert Moss,Have you ever said something was “only a dream...,NaN,Hardcover,9.78158E+12,224 pages,4.04,345,41,"The Three ""Only"" Things: Tapping the Power of ...",Nonfiction|Spirituality|Psychology|Spiritualit...,https://images.gr-assets.com/books/1328746102l...,"[Nonfiction, New Age, Psychology, Spirituality]",[Nonfiction]
16021,David Ambrose,A train of seemingly random events - coinciden...,NaN,Unknown Binding,9.78074E+12,NaN,3.35,166,18,Coincidence,Fiction|Suspense|Historical|Science Fiction|My...,https://images.gr-assets.com/books/1408924887l...,"[Suspense, Fiction, Mystery, Historical, Scien...","[Fiction, Mystery, Historical, Science Fiction]"
49981,Richard North Patterson,When the body of eleven-year-old Thuy Sen is f...,NaN,Hardcover,9.78035E+12,463 pages,3.89,2400,154,Conviction,Fiction|Mystery|Thriller|Legal Thriller|Thriller,https://images.gr-assets.com/books/1399495278l...,"[Thriller, Legal Thriller, Fiction, Mystery]","[Thriller, Fiction, Mystery]"


In [14]:
#download training images
#download_images(train,"train_images")

In [15]:
#download test images
#download_images(test, "test_images")

In [16]:
train_filename_col = return_image_name_col(train)

In [17]:
train_df = train[['genres_cut']]
train_df['file_name'] = train_filename_col

In [18]:
train_df.head()

,genres_cut,file_name
6377,"[Romance, Contemporary]",6377.jpg
19880,"[Fiction, Romance, Historical, Cultural]",19880.jpg
23352,[Nonfiction],23352.jpg
16021,"[Fiction, Mystery, Historical, Science Fiction]",16021.jpg
49981,"[Thriller, Fiction, Mystery]",49981.jpg


In [19]:
test_filename_col = return_image_name_col(test)
test_df = test[['genres_cut']]
test_df['file_name'] = test_filename_col
test_df.head()

,genres_cut,file_name
9102,"[Romance, Fiction]",9102.jpg
34116,"[History, Biography, Nonfiction]",34116.jpg
53879,"[Crime, Fiction, Mystery, Thriller]",53879.jpg
30327,"[Childrens, Fantasy]",30327.jpg
36487,"[Young Adult, Fantasy]",36487.jpg


In [20]:
train_df_valid = train_df.copy()

In [21]:
for index,row in train_df.iterrows():
    path = 'train_images/' + row['file_name']
    if os.stat(path).st_size < 1024:
        train_df_valid.drop(index = index,inplace=True)
    

In [22]:
train_df_valid.shape

(36298, 2)

In [23]:
test_df_valid = test_df.copy()
for index,row in test_df.iterrows():
    path = 'test_images/' + row['file_name']
    if os.stat(path).st_size < 1024:
        test_df_valid.drop(index = index,inplace=True)

In [24]:
test_df_valid.shape

(12096, 2)

In [25]:
#pickle_out = open("train_images_valid.pickle","wb")
#pickle.dump(train_df_valid, pickle_out)
#pickle_out.close()

In [26]:
#pickle_out = open("test_images_valid.pickle","wb")
#pickle.dump(test_df_valid, pickle_out)
#pickle_out.close()

In [27]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

In [28]:
from sklearn.model_selection import KFold

In [29]:
all_classes = np.unique(list(itertools.chain.from_iterable(train_df['genres_cut'])))

In [30]:
kf = KFold(n_splits=5,random_state=None, shuffle=False)

### Model construction

In [31]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [32]:
#bulid the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss=[hn_multilabel_loss],metrics=[f1])

In [33]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [34]:
callbacks = [ ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='model.h5', save_best_only=True)
            ]

### New train feature construction

In [35]:
predictions_lst = []

In [36]:
for train_index, test_index in kf.split(train_df_valid):
    #print(train_index,test_index)
    #train generator
    train_generator=datagen.flow_from_dataframe(
    dataframe=train_df_valid.iloc[train_index],
    directory="train_images",
    x_col="file_name",
    y_col="genres_cut",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=list(all_classes),
    target_size=(100,100))
    
    #test generator 
    test_generator=test_datagen.flow_from_dataframe(
    dataframe=train_df_valid.iloc[test_index],
    directory="train_images",
    x_col="file_name",
    y_col=None,
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(100,100))
    
    #fitting the model
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=15,
                    callbacks=callbacks)
    #prediction
    test_generator.reset()
    pred=model.predict_generator(test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
    
    pred_bool = (pred > 0.5)
    

    predictions = []
    labels = train_generator.class_indices
    labels = dict((v,k) for k,v in labels.items())
    for row in pred_bool:
        l=[]
        for index,cls in enumerate(row):
            if cls:
                l.append(labels[index])
        predictions.append(l)
    predictions_lst.extend(predictions)


Found 29038 validated image filenames belonging to 23 classes.
Found 7260 validated image filenames.
Epoch 1/15
907/907 [==============================] - 486s 535ms/step - loss: 7.9275 - f1: 0.2054
Epoch 2/15
907/907 [==============================] - 518s 571ms/step - loss: 7.5442 - f1: 0.2265
Epoch 3/15
907/907 [==============================] - 562s 620ms/step - loss: 7.3847 - f1: 0.2448
Epoch 4/15
907/907 [==============================] - 577s 636ms/step - loss: 7.2603 - f1: 0.2626
Epoch 5/15
907/907 [==============================] - 559s 617ms/step - loss: 7.1624 - f1: 0.2751
Epoch 6/15
907/907 [==============================] - 548s 604ms/step - loss: 7.0668 - f1: 0.2883
Epoch 7/15
907/907 [==============================] - 458s 505ms/step - loss: 6.9818 - f1: 0.3001
Epoch 8/15
907/907 [==============================] - 455s 502ms/step - loss: 6.8991 - f1: 0.3098
Epoch 9/15
907/907 [==============================] - 455s 501ms/step - loss: 6.8139 - f1: 0.3240
Epoch 10/15
907/9

In [38]:
len(predictions_lst)

36298

In [40]:
new_feature_train = train_df_valid.copy()

In [41]:
new_feature_train['keras_pred'] = predictions_lst

In [43]:
new_feature_train.head()

,genres_cut,file_name,keras_pred
6377,"[Romance, Contemporary]",6377.jpg,"[Fantasy, Paranormal, Romance]"
19880,"[Fiction, Romance, Historical, Cultural]",19880.jpg,[Fiction]
23352,[Nonfiction],23352.jpg,[Fiction]
16021,"[Fiction, Mystery, Historical, Science Fiction]",16021.jpg,[Fiction]
49981,"[Thriller, Fiction, Mystery]",49981.jpg,[Fiction]


In [44]:
pickle_out = open("train_with_new_feature.pickle","wb")
pickle.dump(new_feature_train, pickle_out)
pickle_out.close()

### New test feature construction

In [55]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df_valid,
directory="train_images",
x_col="file_name",
y_col="genres_cut",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=list(all_classes),
target_size=(100,100))

Found 36298 validated image filenames belonging to 23 classes.


In [56]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df_valid,
directory="test_images",
x_col="file_name",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(100,100))

Found 12096 validated image filenames.


In [57]:
#bulid the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss=[hn_multilabel_loss],metrics=[f1])

In [58]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [59]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [60]:
callbacks = [ ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='model-simple2.h5', save_best_only=True)
            ]

In [61]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=15,
                    callbacks=callbacks
)

Epoch 1/15
1134/1134 [==============================] - 673s 594ms/step - loss: 7.8354 - f1: 0.2092
Epoch 2/15
1134/1134 [==============================] - 694s 612ms/step - loss: 7.4744 - f1: 0.2400
Epoch 3/15
1134/1134 [==============================] - 667s 588ms/step - loss: 7.3093 - f1: 0.2628
Epoch 4/15
1134/1134 [==============================] - 724s 638ms/step - loss: 7.1994 - f1: 0.2758
Epoch 5/15
1134/1134 [==============================] - 665s 586ms/step - loss: 7.0796 - f1: 0.2939
Epoch 6/15
1134/1134 [==============================] - 668s 589ms/step - loss: 6.9800 - f1: 0.3045
Epoch 7/15
1134/1134 [==============================] - 610s 537ms/step - loss: 6.8964 - f1: 0.3140
Epoch 8/15
1134/1134 [==============================] - 569s 502ms/step - loss: 6.8196 - f1: 0.3253
Epoch 9/15
1134/1134 [==============================] - 568s 501ms/step - loss: 6.7528 - f1: 0.3338
Epoch 10/15
1134/1134 [==============================] - 568s 501ms/step - loss: 6.6860 - f1: 0.3451

In [62]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

12096/12096 [==============================] - 184s 15ms/step


In [81]:
pred[0]

array([6.1069563e-04, 3.5543285e-02, 5.2862433e-03, 1.9549245e-02,
       3.4089231e-01, 1.2600668e-03, 3.9944261e-02, 7.8190360e-03,
       1.0479687e-02, 2.2908087e-01, 1.7063333e-01, 2.6924433e-02,
       2.2126318e-04, 3.2485332e-02, 1.0209440e-02, 1.7553844e-02,
       4.1463187e-01, 1.2642478e-02, 1.8559820e-03, 5.3337640e-01,
       3.6512255e-03, 2.1477186e-03, 1.9528612e-02], dtype=float32)

In [82]:
pred_bool = (pred > 0.5)

In [83]:
#predictions=[]
predictions2 = []
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions2.append(l)
#    predictions.append(",".join(l))
#filenames=test_generator.filenames
#results=pd.DataFrame({"Filename":filenames,
#                      "Predictions":predictions})
#results.to_csv("results_keras.csv",index=False)

In [84]:
new_feature_test = test_df_valid.copy()

In [85]:
new_feature_test['keras_pred'] = predictions2
new_feature_test.head()

,genres_cut,file_name,keras_pred
9102,"[Romance, Fiction]",9102.jpg,[Romance]
34116,"[History, Biography, Nonfiction]",34116.jpg,[]
53879,"[Crime, Fiction, Mystery, Thriller]",53879.jpg,[]
30327,"[Childrens, Fantasy]",30327.jpg,[]
36487,"[Young Adult, Fantasy]",36487.jpg,[Fiction]


In [86]:
pickle_out = open("test_with_new_feature.pickle","wb")
pickle.dump(new_feature_train, pickle_out)
pickle_out.close()